## Bias and efficiency
The focus of these simulations is to show how altering a model, to avoid collinear regressors, can result in biased estimates of parameter estimates. Although efficiency is preferred, it is less important than bias in terms of interpreting contrast estimates.

An fMRI model should include a regressor for each stimulus that occurs.  Since the cue/fixation/probe/feedback stimuli are presented without any baseline fixation between them, modeling all stimuli is often avoided, presumably in fear that the lowered efficiency will reduce power.  The less appreciated, more important issue is the bias that results when this modeling practice is used.

NOTE: This version is refactored by RP based on the code in MID_sim_groupmodel_manuscript.ipynb. Changes include:

- removing the fix only model (since it's not directly relevant to the point of the paper)
- moving all of the functions to external files
- testing of all functions (`make test`)


In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

from tqdm import tqdm
from scipy.stats import ttest_1samp
import pickle
from collections import OrderedDict
from nilearn.plotting import plot_design_matrix

from simulation_funcs import (
    get_subdata_long,
    insert_jitter,
    create_design_matrices,
    generate_data_nsim,
    get_beta_dicts,
    create_contrasts,
    make_analysis_label,
    sim_group_models_parallel,
)

from simulation_plotting import (
    plot_proportion_sig,
    plot_contrast_estimates,
    plot_results,
    plot_dict_of_results,
)

## Simulation setup

These simulations focus on the following models:
* saturated:  Models 5 cue regressors (2 levels of gain/loss and no money at stake), 5 similar fixation regressors, 5 probe regressors (when RT is unknown) and 10 feedback regressors (5 trial types x hit/miss)
* cue only: Model includes 5 cue regressors and 10 feedback regressors
* cuefix: Model includes 5 regressors using the cue onset and cue+fixation durations and 10 feedback regressors
* fix only: Model includes 5 fixation regressors and 10 feedback regressors

The models for  one subject are displayed below


In [ ]:
sub = 1
events = get_subdata_long(sub)
designs = create_design_matrices(events, conv_resolution=0.2, tr=1)
print(designs.keys())

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15, 20))
ax_flat = ax.flatten()
for i, desname in enumerate(designs.keys()):
    reg_names = designs[desname].columns
    trial_types = [
        val.replace('FEEDBACK_HIT_', '')
        for val in reg_names
        if 'FEEDBACK_HIT_' in val
    ]
    stim_types = [
        val.replace('_LargeLoss', '')
        for val in reg_names
        if 'LargeLoss' in val
    ]
    regressors_ordered = [
        f'{stim_type}_{trial_type}'
        for trial_type in trial_types
        for stim_type in stim_types
    ]
    plot_design_matrix(
        designs[desname].reindex(regressors_ordered, axis=1), ax=ax_flat[i]
    )
    ax_flat[i].set_title(desname)
plt.show()

### The main contrasts of interest
* Anticipation: Win - Neutral
* Anticipation: Large Win - Neutral
* Feedback: WinHit - Neutral Miss
* Feedback: Large Win Hit - Large Win Miss
The definitions for each of these contrasts for each model are given below.

In [ ]:
contrast_strings, contrasts_matrices, c_pinv_xmats = create_contrasts(designs)
for key, contrasts in contrast_strings.items():
    print(f'\n Contrasts for {key} model')
    for contrast_name, contrast_string in contrasts.items():
        if ':' in contrast_name:
            print(f'{contrast_name}: {contrast_string}')

### Effect sizes and variance settings
The model used to simulate data was the saturated model.  The null model (all true betas were set to 0) and 6 non-null models were simulated, where the nonzero betas are define below in `beta_dicts`. It is hard to know what a reasonable within- and between-subject variance would be.  We simply set the between-subject variance to 1 and the within-subject residual variance was set to 1.  As such, we cannot make any claims about power differences here, since that would require properly setting the variances to realistic values based on data.  Instead, we have opted to run the model comparisons on real data to show any power differences, barring the presence of bias.

A totall of 1000 simulated data sets were generated.  Each data set has 107 subjects and the timings are based on the AHRB data.  Data were simulated, with the given beta settings and then contrasts were estimated using each model.  The contrast estimates were then averaged over subjects and the inference from the 1-sample t-test was used to determine significance using a 2-tailed test p-value < 0.05.

In [ ]:
# Run all simulations without jitter
#  Note, the error for subject 5 is due to a problem with their file, so they have been omitted from the simulations.
beta_dicts = get_beta_dicts()

beta_sub_sd = 1
noise_sd = 1
nsims = 1000
results = {}

for beta_dict in beta_dicts:
    figure_label = make_analysis_label(beta_dict)
    results[figure_label], _ = sim_group_models_parallel(
        beta_dict,
        noise_sd,
        beta_sub_sd,
        nsims=nsims,
        conv_resolution=0.2,
        tr=1,
    )

## Type I errors are controlled when there is no signal
The following figure show the proportion of significant results out of the 1000 simulated data sets and the distributions of the group contrast estimates.  Bars and violins that are opaque (all in the this figure) reflect contrasts that should be null.  In the left panel (Proportion significant) all bars reflect Type I error rates and these are controlled for all contrasts and models.  In the right panel, since all contrasts reflect null effects, the distributions of the coefficient estimates should be, and are, centered about 0.

In [ ]:
plot_dict_of_results({'Null model': results['Null model']}, contrasts=True)

## Parsing the results
Error rates, power and bias for the four contrasts of interest. Each panel grouping in the following figure refers to different settings where some parameters are nonzero as indicated in the panel pair figure headings.  Solid colored bars/violins reflect contrasts that should have signal, while opaque reflects null models.  If an opaque bar is above 0.05 (dashed red line), this indicates an inflated Type I error for that model and contrast combination.  The corresponding distributions of coefficient estimates will reflect the bias that occurs when the error rate is inflated.  Notably, bias can occur for null and non-null effects.

When both the small and large gain cue regressors have nonzero betas (=0.4), the fix only model cannot fully capture the cue effect that is present and this left over signal biases the feedback estimates, which inflates the type I error. In the cuefix model, since the model does not properly fit the signal (the duration of the regressor is too long), since the cuefix regressor is collinear with the feedback regressors, this ends up biasing the feedback estimate and inflating the...

In [ ]:
results_plot = results.copy()
results_plot.pop('Null model')
plot_dict_of_results(results_plot, contrasts=True)

In [ ]:
for name, result in results.items():
    plot_results(
        result[(result['contrast'].str.contains(':') == False)],
        name,
        stacked=True,
    )

## Repeat the above, but add a jittered ITI

These simulations add a jitter between 2-5s between the offset of the feedback of one trial and the onset of the cue in the following trial.

In [ ]:
results_jitter = {}

for beta_dict in beta_dicts:
    figure_label = make_analysis_label(beta_dict)
    results_jitter[figure_label], _ = sim_group_models_parallel(
        beta_dict,
        noise_sd,
        beta_sub_sd,
        nsims=nsims,
        conv_resolution=0.2,
        tr=1,
        jitter=False,
        jitter_iti_min=2,
        jitter_iti_max=4,
    )

In [ ]:
results_jitter_plot = results_jitter.copy()
results_jitter_plot.pop('Null model')
plot_dict_of_results(results_jitter_plot, contrasts=True)

In [ ]:
for name, result in results_jitter.items():
    plot_results(
        result[(result['contrast'].str.contains(':') == False)],
        name,
        stacked=True,
    )